In [13]:
!pip install -q -U datasets
!pip install -q -U transformers

In [47]:
import glob 
from datasets import load_dataset

def loadDataSet():
    text_files = glob.glob('../trainData/*train_data.csv')
    dataset_files = text_files
    # dataset_name = "../trainData/*train_data.csv"
    dataset = load_dataset("csv", data_files=dataset_files, split="train")
    return dataset

In [49]:

def form_data(example):    
    test_str2 = example['Tamil Answer']
    test_str1 = example['Tamil Question']
    if (test_str1 is None): 
        test_str1 = ""
    data = test_str1 + " \n" + test_str2
    data
    return data
    
def isRowValid(example):
    test_str2 = example['Tamil Answer']
    test_str1 = example['Tamil Question']
    if (example is None) or (test_str2 is None) or (len(test_str2.strip()) == 0):
        return False
    if (example is None) or (test_str1 is None) or (len(test_str1.strip()) == 0):
        return False
    return True
        
def updatedDataset(dataset):
    new_column = []
    for x in dataset:
        new_column.append(form_data(x))
    dataset = dataset.add_column('data', new_column)        
    return dataset

def getDataSet():
    dataset = loadDataSet()
    print ( "total rows in dataset: "+ str(len(dataset)))
    filtered_dataset = dataset.filter(isRowValid)    
    filtered_dataset = updatedDataset(filtered_dataset)
    print ( "total rows in dataset after filtering: "+ str(len(filtered_dataset)))
    return filtered_dataset

dataset = getDataSet()

Generating train split: 0 examples [00:00, ? examples/s]

total rows in dataset: 2236


Filter:   0%|          | 0/2236 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/2012 [00:00<?, ? examples/s]

total rows in dataset after filtering: 2012


In [51]:
def apply_chat_template(
    example,
    # tokenizer,
):
    # messages = example["messages"]
    test_str2 = example['Tamil Answer']
    test_str1 = example['Tamil Question']
    data = []
    data.append({"role": "system", "content": ""})
    data.append({"role": "user", "content": test_str1})
    data.append({"role": "assistant", "content": test_str2})
    
    # Add an empty system message if there is none
    # if messages[0]["role"] != "system":
    #     messages.insert(0, {"role": "system", "content": ""})
    # example["text"] = tokenizer.apply_chat_template(
    #     messages, tokenize=False, add_generation_prompt=False)
    example["text"] = data
    return example

processed_train_dataset = dataset.map(
    apply_chat_template,
    #fn_kwargs={"tokenizer": tokenizer},
    #num_proc=10,
    #remove_columns=column_names,
    desc="Applying chat template to train_sft",
)
processed_train_dataset

Applying chat template to train_sft:   0%|          | 0/2012 [00:00<?, ? examples/s]

Dataset({
    features: ['English Question', 'Tamil Question', 'Tamil Answer', 'data', 'text'],
    num_rows: 2012
})

In [53]:
processed_train_dataset['Tamil Answer'][0:5]

['எழுத்துக்கள் எல்லாம் அகரத்தை அடிப்படையாக கொண்டிருக்கின்றன. அதுபோல உலகம் கடவுளை அடிப்படையாக கொண்டிருக்கிறது.',
 ' மழை பெய்ய உலகம் வாழ்ந்து வருவதால், மழையானது உலகத்து வாழும் உயிர்களுக்கு அமிழ்தம் என்று உணரத்தக்கதாகும்',
 'ஒழுக்கத்தில் நிலைத்து நின்று பற்று விட்டவர்களின் பெருமையைச் சிறந்ததாக போற்றி கூறுவதே நூல்களின் துணிவாகும்.',
 'அறம் சிறப்பையும் அளிக்கும்: செல்வத்தையும் அளிக்கும்: ஆகையால் உயிர்க்கு அத்தகைய அறத்தை விட நன்மையானது வேறு யாது?',
 'இல்லறத்தில் வாழ்பவனாகச் சொல்லப்படுகிறவன் அறத்தின் இயல்பை உடைய மூவருக்கும் நல்வழியில் நிலை பெற்ற துணையாவான்.']

In [61]:
processed_train_dataset['text'][2000:2012]

[[{'content': '', 'role': 'system'},
  {'content': 'திருக்கோவையாருக்கு வழங்கப்பட்ட அடைமொழி என்ன?', 'role': 'user'},
  {'content': 'இராசாக் கோவை', 'role': 'assistant'}],
 [{'content': '', 'role': 'system'},
  {'content': 'குறவஞ்சிக்கு வழங்கப்பட்ட அடைமொழிகள் என்னென்ன?', 'role': 'user'},
  {'content': 'குறம், குறவஞ்சி நாடகம், குறவஞ்சி நாட்டியம்',
   'role': 'assistant'}],
 [{'content': '', 'role': 'system'},
  {'content': 'உலாவுக்கு வழங்கப்பட்ட அடைமொழி என்ன?', 'role': 'user'},
  {'content': 'உலாப்புறம்', 'role': 'assistant'}],
 [{'content': '', 'role': 'system'},
  {'content': 'தாயுமானவர் பாடல்களுக்கு வழங்கப்பட்ட அடைமொழி என்ன?',
   'role': 'user'},
  {'content': 'தமிழ்மொழியின் உபநிடதம்', 'role': 'assistant'}],
 [{'content': '', 'role': 'system'},
  {'content': '"பிரித்தெழுதுக" என்ற கருத்து தமிழில் என்ன?', 'role': 'user'},
  {'content': '"பிரித்தெழுதுக" என்பது ஒரு சொல்லை அதை உருவாக்கும் தனித்தனி எழுத்துகளாக பிரிக்கும் செயல்முறையைக் குறிக்கிறது. இது தமிழ் மொழி கற்றல் மற்றும் TNPSC போன்ற போட

'\nsdfsdfs\nsdfsdf\nfsf\n'